In [46]:
#setting up the DB
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../data/nobel_prize.db', echo=True) # where is db


from sqlalchemy.ext.declarative import declarative_base #This you have to declare your own schemas like NoSQL and MongoDB
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Enum

class winnerTable(Base): #a child class of Base object
    __tablename__ = 'winner'
    
    #defining the columns
    id = Column(Integer, primary_key=True) 
    category = Column(String)
    name = Column(String)
    nationality = Column(String)
    sex = Column(Enum('male','female'))
    year = Column(Integer)
    
    def __repr__(self):
        return "<winnerTable(name='%s',nationaliy ='%s', year='%d', sex='%s')" %(self.name, self.category, self.year, self.sex)

Base.metadata.create_all(engine)

2018-10-13 05:02:28,834 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-10-13 05:02:28,834 INFO sqlalchemy.engine.base.Engine ()
2018-10-13 05:02:28,834 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-10-13 05:02:28,835 INFO sqlalchemy.engine.base.Engine ()
2018-10-13 05:02:28,836 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("winner")
2018-10-13 05:02:28,837 INFO sqlalchemy.engine.base.Engine ()
2018-10-13 05:02:28,840 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE winner (
	id INTEGER NOT NULL, 
	category VARCHAR, 
	name VARCHAR, 
	nationality VARCHAR, 
	sex VARCHAR(6), 
	year INTEGER, 
	PRIMARY KEY (id), 
	CHECK (sex IN ('male', 'female'))
)


2018-10-13 05:02:28,840 INFO sqlalchemy.engine.base.Engine ()
2018-10-13 05:02:28,854 INFO sqlalchemy.engine.base.Engine COMMIT


In [54]:
#parsing json to a DB

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

import json
with open('../data/nobel_winners.json') as jsonfile:
    reader = json.load(jsonfile)
    for row in reader:
        session.add(winnerTable(**row)) #double stars important, unpacks data
    session.commit()
    
    

2018-10-13 05:10:35,229 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-10-13 05:10:35,230 INFO sqlalchemy.engine.base.Engine INSERT INTO winner (category, name, nationality, sex, year) VALUES (?, ?, ?, ?, ?)
2018-10-13 05:10:35,231 INFO sqlalchemy.engine.base.Engine ('A', 'A name', 'A nation', 'male', 1)
2018-10-13 05:10:35,242 INFO sqlalchemy.engine.base.Engine INSERT INTO winner (category, name, nationality, sex, year) VALUES (?, ?, ?, ?, ?)
2018-10-13 05:10:35,243 INFO sqlalchemy.engine.base.Engine ('B', 'B name', 'B nation', 'female', 2)
2018-10-13 05:10:35,244 INFO sqlalchemy.engine.base.Engine INSERT INTO winner (category, name, nationality, sex, year) VALUES (?, ?, ?, ?, ?)
2018-10-13 05:10:35,245 INFO sqlalchemy.engine.base.Engine ('C', 'C name', 'C nation', 'male', 3)
2018-10-13 05:10:35,246 INFO sqlalchemy.engine.base.Engine COMMIT


In [58]:
session.query(winnerTable).count()

result = session.query(winnerTable).filter_by(nationality='A nation')
list(result)

2018-10-13 05:12:14,182 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT winner.id AS winner_id, winner.category AS winner_category, winner.name AS winner_name, winner.nationality AS winner_nationality, winner.sex AS winner_sex, winner.year AS winner_year 
FROM winner) AS anon_1
2018-10-13 05:12:14,183 INFO sqlalchemy.engine.base.Engine ()
2018-10-13 05:12:14,185 INFO sqlalchemy.engine.base.Engine SELECT winner.id AS winner_id, winner.category AS winner_category, winner.name AS winner_name, winner.nationality AS winner_nationality, winner.sex AS winner_sex, winner.year AS winner_year 
FROM winner 
WHERE winner.nationality = ?
2018-10-13 05:12:14,186 INFO sqlalchemy.engine.base.Engine ('A nation',)


[<winnerTable(name='A name',nationaliy ='A', year='1', sex='male'),
 <winnerTable(name='A name',nationaliy ='A', year='1', sex='male'),
 <winnerTable(name='A name',nationaliy ='A', year='1', sex='male')]